# **Streamlit**

NB support pour le développement de l'API + Dashboard, directement en apprenant Streamlit + FastAPI.

Méthode :
1. m'inscrire strictement dans la spec
2. remettre en place les fonctions d’extraction de données (la façade du back office = l’API in fine, avec une route pour chaque requête).
3. de me familiariser avec chacun des composants graphiques de streamlit

# Spec : Dashboard et déploiement

**Note perso sur ma v1** échec en raison d'une erreur de conception critique en amont (non couplage en pipeline de (scaler, imputer) et classifier).

Il est attendu de réaliser 2 applications :
* Une API qui au minimum renvoie le score d’un client (moteur d’inférence qui réalise le predict à partir du modèle final sauvegardé)
* Une application Dashboard qui visualise les informations du client et les graphiques associés, et qui appelle l’API via une url pour récupérer le score du client
* Ces applications seront gérées avec un logiciel de gestion de version comme Git (faire des commit), et l’ensemble du code déposé sur Github via push à partir de Git
* L’API peut être réalisée par exemple en Flask (conseillé) ou Django (plus complexe)
* La réalisation technique du Dashboard peut s’appuyer sur les librairies Dash ou Bokeh, ainsi qu’avec Streamlit qui met également à disposition un serveur cloud pour exécuter l’application
* Les applications (Dashboard et l’API) seront hébergées sur un serveur cloud, idéalement sur un serveur gratuit Azure webapp, AWS, PythonAnywhere, Heroku, Streamlit.

Recommandations :
* API
    * Corrigé du webinaire : déployez une API de prédiction
    * L’API peut être réalisée par exemple en Flask (conseillé) ou Django (plus complexe)
* Dashboard
    * La réalisation technique du Dashboard peut s’appuyer sur les librairies Dash ou Bokeh, ainsi qu’avec Streamlit qui met également à disposition un serveur cloud pour exécuter l’application
    * Spec du dashboard (perdue au milieu des milestones) :
        * N° client (analyse : donc le SK_ID_CURR), crédit accepté ou non (analyse : donc la décision du classifieur), score détaillé sous forme de jauge colorée selon qu’il est en dessous ou au-dessus du seuil : permet de juger s’il est loin du seuil ou non (anayse : donc la prédiction probabiliste du classifieur).
        * Sa feature importance locale sous forme de graphique, qui permet au chargé d’étude de comprendre quelles sont les données du client qui ont le plus influencé le calcul de son score (analyse : besoin de SHAP, mais je ne sais pas vraiment comment l’utiliser)
        * Le tableau de bord présentera également d’autres graphiques sur les autres clients :
            * Deux graphiques de features sélectionnées dans une liste déroulante, présentant la distribution de la feature selon les classes, ainsi que le positionnement de la valeur du client
            * Un graphique d’analyse bi-variée entre les deux features sélectionnées, avec un dégradé de couleur selon le score des clients, et le positionnement du client
            * La feature importance globale
* Déploiement sur Azure :
    * https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Ing%C3%A9nieur_IA_P7/Seuils_gratuite%CC%81_Azure.pdf
    * Les applications (Dashboard et l’API) seront hébergées sur un serveur cloud, idéalement sur un serveur gratuit Azure webapp, AWS, PythonAnywhere, Heroku, Streamlit.
    * Analyse : donc au pire, cloud de Streamlit, si je n’arrive pas à déployer sur Azur.

# Journal d'avancement projet

## 04/09/2023

Le but est de développer aussi vite que possible une première version de mon application finale basée sur streamlit, laquelle interroge des API FastAPI. Pour bien découpler les choses et avancer par itérations, je vais m'appuyer d'abord sur un dummy model de prédiction, voire pas de prédiction du tout, mais le renvoi soit de la classe connue du client partant de l'ensemble d'apprentissage, soit un None en attendant la mise à disposition du modèle prédictif.

Ainsi, je peux déjà faire le lien entre les données pré-traitées et l'interface de l'utilisateur final.

Cela me permet d'aller jusqu'au déploiement sur un cloud (Streamlit, ou un hébergement gratuit quelconque) de cette application pour la tester dans la situation de déploiement final.

---

La première chose, c'est que les données à disposition sont de plusieurs version :
1. les données brutes non retraitées, distribuées dans 7 tables.
2. les données nettoyées, retraitées, complétées, toujours dans les 7 tables.
3. les données dans les tables fusion (groupby), sur plusieurs niveaux, jusqu'à la table intégrée de tout avec 3.1/ méthode de fusion d'un kernel par défaut, 3.2/ méthode de fusion de mon kernel amélioré (qu'il faut retravailler).
4. avant ou après one_hot_encoding, lequel démultiplie artificiellement les caractéristiques catégorielles.

Je pense que pour y aller doucement, par petits pas, en intégrant ensuite les parties, nous allons commencer par mettre en place une interface simple pour charger et afficher une table parmis les différentes options exposées ci-dessus, et aider à filtrer son contenu, notamment d'abord à l'aide du SK_ID_CURR qui identifie un client.

# Récupération des données sur un client

Deux approches possibles : taper dans les tables ciblées vs. dans la version intégrée monotable (dépend de mon retour sur RLE, un sous-chantier passionnant)

# Liste déroulante des caractéristiques

Objectifs :
1. Seulement la liste déroulante avec toutes les caractéristiques.
2. Associer sélection et affichage : distribution de la caractéristique selon les classes
3. Ajout des informations de mon help sur les caractéristiques : FR / EN
4. Ajout de la possibilité de filtrer par famille de caractéristiques, et par type => réduction de la liste de sélection

Pour cela, il me faut des fonctions (à wrapper ensuite en APIs) + des composants graphiques.

Commençons donc par les fonctions.

## Liste des caractéristiques

Quelle est la différence entre mes deux fonctions (qui n'ont pas été documentées)?

Il serait utile que la seconde propose un certain nombre de méta-données qui enrichissent leur description.

In [1]:
from home_credit.load import get_columns_description
col_descs = get_columns_description()
display(col_descs)

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\HomeCredit_columns_description.pqt


col_descs,Table,Column,Description,Special
col_id,,,,
1,application_{train|test},SK_ID_CURR,ID of loan in our sample,None
2,application_{train|test},TARGET,Target variable (1 - client with payment diffi...,None
5,application_{train|test},NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,None
6,application_{train|test},CODE_GENDER,Gender of the client,None
7,application_{train|test},FLAG_OWN_CAR,Flag if the client owns a car,None
...,...,...,...,...
217,installments_payments,NUM_INSTALMENT_NUMBER,On which installment we observe payment,None
218,installments_payments,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application


In [1]:
from home_credit.load import get_var_descs
var_descs = get_var_descs()
display(var_descs)

load C:\Users\franc\Projects\pepper_credit_scoring_tool\dataset\pqt\HomeCredit_columns_description.pqt


col_descs,Table,Column,Description,Special
col_id,,,,
1,application,SK_ID_CURR,ID of loan in our sample,None
2,application,TARGET,Target variable (1 - client with payment diffi...,None
5,application,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,None
6,application,CODE_GENDER,Gender of the client,None
7,application,FLAG_OWN_CAR,Flag if the client owns a car,None
...,...,...,...,...
217,installments_payments,NUM_INSTALMENT_NUMBER,On which installment we observe payment,None
218,installments_payments,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application


Streamlit selectbox

Voir `feat_selectbox_1.py`

```python
feature = st.selectbox("Select a feature", features)
```

In [4]:
features = var_descs.Table.str.cat(var_descs.Column, sep=".")
display(features)

col_id
1                           application.SK_ID_CURR
2                               application.TARGET
5                   application.NAME_CONTRACT_TYPE
6                          application.CODE_GENDER
7                         application.FLAG_OWN_CAR
                          ...                     
217    installments_payments.NUM_INSTALMENT_NUMBER
218          installments_payments.DAYS_INSTALMENT
219       installments_payments.DAYS_ENTRY_PAYMENT
220           installments_payments.AMT_INSTALMENT
221              installments_payments.AMT_PAYMENT
Name: Table, Length: 219, dtype: object